In [2]:
INPUT = "/Users/joregan/Playing/irish-gists/wiktionary/2025-extract.txt"

In [3]:
with open(INPUT) as f:
    text = f.read()
pieces = text.split("\n---\n")

In [6]:
pieces = [p.strip() for p in pieces if p != ""]

In [9]:
pieces = [p for p in pieces if "|ga|" in p]

In [11]:
quiggin = [p for p in pieces if "Quiggin" in p]

In [15]:
sep_quote = [p for p in quiggin if "* {{R:ga:Quiggin" in p]

In [ ]:
sep_quote

In [24]:
import urllib
extracted = []
for p in sep_quote:
    lines = p.split("\n")
    word = lines[0].strip()
    id = lines[1].strip().replace("ID ", "")
    lines = lines[2:]
    if len(lines) == 2:
        raw = lines
    else:
        raw = []
        for line in lines:
            if line.startswith("* {{R:ga:Quiggin"):
                raw.append(line)
            elif "Ulster" in line:
                if "<comment>" in line:
                    continue
                raw.append(line)
    data = {}
    data["irish"] = word
    url_word = urllib.parse.quote(word.replace(" ", "_"))
    data["url"] = f"https://en.wiktionary.org/w/index.php?title={url_word}&oldid={id}"
    data["raw"] = raw
    extracted.append(data)

In [ ]:
extracted